Model Description:
Joint Multi-Asset Cross-Asset Factor Model (HYG, LQD, TLT, SPY)
- Builds aligned returns for 4 ETFs
- Builds a shared cross-asset factor set
- Runs OLS per asset (same X across assets)
- Runs PCA on factors per frequency and regressions on PCs
- Frequencies: Daily, Weekly (Friday), Monthly (month-end)

Data description:
- Workshop Data.xlsx:
    - sheet "Portfolio" contains: Date, {SPY,HYG,LQD,TLT} Position, {SPY,HYG,LQD,TLT} Dividends
    - sheets "Adj HYG", "Adj LQD" contain: Date, Volume
    - sheets "SPY", "TLT" contain: Date, Volume (as you referenced)
- Indexes and Spreads Data 01.09.xlsx:
    - sheet "HY Index": Date, OAS_SOVEREIGN_CURVE
    - sheet "IG Index": Date, OAS_SOVEREIGN_CURVE
- Indexes and Spreads Data.xlsx:
    - sheet "UST Index": Date, PX_LAST (LUATTRUU Index proxy)
    - sheet "10yUST Yields": Date, PX_LAST (USGG10YR Index)
- MOVE Vix prices.xlsx:
    - sheet "VIX": Date, PX_LAST   (or "Vix" depending on your file)

In [6]:
# ============================================================
# Imports
# ============================================================

import warnings
warnings.filterwarnings("ignore")

from pathlib import Path
import numpy as np
import pandas as pd
import statsmodels.api as sm

from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA


# ============================================================
# CONFIG (edit as needed)
# ============================================================

CONFIG = {
    "DATA_DIR": None,

    "FILES": {
        "workshop": "Workshop Data.xlsx",
        "spreads_0109": "Indexes and Spreads Data 01.09.xlsx",
        "spreads_main": "Indexes and Spreads Data 01.09.xlsx",
        "vix": "MOVE Vix prices.xlsx",
    },

    "SHEETS": {
        "portfolio": "Portfolio",

        # Volumes
        "adj_hyg": "Adj HYG",   # Date, Volume
        "adj_lqd": "Adj LQD",   # Date, Volume
        "spy_vol": "SPY",       # Date, Volume
        "tlt_vol": "TLT",       # Date, Volume

        # Spreads
        "hy_index": "HY Index",  # Date, OAS_SOVEREIGN_CURVE
        "ig_index": "IG Index",  # Date, OAS_SOVEREIGN_CURVE

        # Rates / UST indexes
        "ust_index": "UST Index",       # Date, PX_LAST  (LUATTRUU proxy)
        "ust10_yields": "10yUST Yields", # Date, PX_LAST  (USGG10YR yield)

        # Vol index
        "vix": "VIX",  # Date, PX_LAST
    },

    "COLS": {
        "date": "Date",
        "volume": "Volume",
        "px_last": "PX_LAST",
        "oas": "OAS_SOVEREIGN_CURVE",
    },

    # Frequencies (weekly on Friday)
    "FREQUENCIES": {
        "Daily": "D",
        "Weekly": "W-FRI",
        "Monthly": "M",
    },

    # PCA controls
    "PCA_MAX_COMPONENTS": 5,
    "PCA_USE_K": 3,  # PCs used in PC regressions
}


# ============================================================
# Helpers: paths & IO
# ============================================================

def _resolve_path(filename: str) -> Path:
    if CONFIG["DATA_DIR"]:
        return Path(CONFIG["DATA_DIR"]) / filename
    return Path(filename)

def load_excel_df(filename: str, sheet: str) -> pd.DataFrame:
    path = _resolve_path(filename)
    if not path.exists():
        raise FileNotFoundError(f"File not found: {path}")
    df = pd.read_excel(path, sheet_name=sheet)
    if CONFIG["COLS"]["date"] not in df.columns:
        raise ValueError(f"'{CONFIG['COLS']['date']}' column missing in file={path.name} sheet={sheet}")
    df[CONFIG["COLS"]["date"]] = pd.to_datetime(df[CONFIG["COLS"]["date"]])
    df = df.set_index(CONFIG["COLS"]["date"]).sort_index()
    return df

def load_excel_series(filename: str, sheet: str, column: str) -> pd.Series:
    df = load_excel_df(filename, sheet)
    if column not in df.columns:
        raise ValueError(
            f"Column '{column}' missing in file={filename} sheet={sheet}. "
            f"Available columns={list(df.columns)}"
        )
    s = pd.to_numeric(df[column], errors="coerce")
    s.name = column
    return s.sort_index()


# ============================================================
# Helpers: transforms & cleaning
# ============================================================

def safe_diff(series: pd.Series) -> pd.Series:
    s = pd.to_numeric(series, errors="coerce")
    return s.diff()

def pct_return_from_level(series: pd.Series) -> pd.Series:
    s = pd.to_numeric(series, errors="coerce")
    return s.pct_change()

def safe_log_return(series: pd.Series, min_positive: float = 1.0) -> pd.Series:
    """
    log(series).diff() but safe for zeros/negatives:
    - coerce non-numeric to NaN
    - clip to min_positive so log() is finite
    """
    s = pd.to_numeric(series, errors="coerce")
    s = s.clip(lower=min_positive)
    return np.log(s).diff()

def to_numeric_df(df: pd.DataFrame) -> pd.DataFrame:
    out = df.copy()
    for c in out.columns:
        out[c] = pd.to_numeric(out[c], errors="coerce")
    return out

def clean_for_model(df: pd.DataFrame) -> pd.DataFrame:
    """
    Removes NaNs and +/-inf. Ensures numeric.
    """
    df = to_numeric_df(df)
    df = df.replace([np.inf, -np.inf], np.nan)
    df = df.dropna()
    return df

def resample_level(series: pd.Series, freq: str, how: str = "last") -> pd.Series:
    if how == "last":
        return series.resample(freq).last()
    if how == "sum":
        return series.resample(freq).sum()
    raise ValueError("how must be 'last' or 'sum'")

def make_total_return_level(portfolio: pd.DataFrame, position_col: str, dividend_col: str, out_name: str) -> pd.Series:
    if position_col not in portfolio.columns:
        raise ValueError(f"Missing '{position_col}' in Portfolio sheet.")
    if dividend_col not in portfolio.columns:
        raise ValueError(f"Missing '{dividend_col}' in Portfolio sheet.")
    pos = pd.to_numeric(portfolio[position_col], errors="coerce")
    div = pd.to_numeric(portfolio[dividend_col], errors="coerce").fillna(0.0)
    cum_div = div[::-1].cumsum()[::-1]
    tr_level = (pos + cum_div).rename(out_name)
    return tr_level


# ============================================================
# Build returns for each asset (same frequency)
# ============================================================

def build_asset_returns(portfolio_df: pd.DataFrame, freq: str) -> pd.DataFrame:
    tr_spy = make_total_return_level(portfolio_df, "SPY Position", "SPY Dividends", "TR_SPY")
    tr_hyg = make_total_return_level(portfolio_df, "HYG Position", "HYG Dividends", "TR_HYG")
    tr_lqd = make_total_return_level(portfolio_df, "LQD Position", "LQD Dividends", "TR_LQD")
    tr_tlt = make_total_return_level(portfolio_df, "TLT Position", "TLT Dividends", "TR_TLT")

    r_spy = pct_return_from_level(resample_level(tr_spy, freq, "last")).rename("SPY")
    r_hyg = pct_return_from_level(resample_level(tr_hyg, freq, "last")).rename("HYG")
    r_lqd = pct_return_from_level(resample_level(tr_lqd, freq, "last")).rename("LQD")
    r_tlt = pct_return_from_level(resample_level(tr_tlt, freq, "last")).rename("TLT")

    return pd.concat([r_spy, r_hyg, r_lqd, r_tlt], axis=1)


# ============================================================
# Build shared cross-asset factors (same frequency)
# ============================================================

def build_common_factors(freq: str) -> pd.DataFrame:
    F = CONFIG["FILES"]
    S = CONFIG["SHEETS"]
    C = CONFIG["COLS"]

    # --- Vol factor: ΔVIX ---
    vix_px = load_excel_series(F["vix"], S["vix"], C["px_last"])
    vix_rs = resample_level(vix_px, freq, "last")
    vix_change = safe_diff(vix_rs).rename("VIX_Change")

    # --- Rates factor: Δ10Y yield (USGG10YR) ---
    ust10 = load_excel_series(F["spreads_main"], S["ust10_yields"], C["px_last"])
    ust10_rs = resample_level(ust10, freq, "last")
    d_10y = safe_diff(ust10_rs).rename("Delta_10Y")

    # --- Treasury market factor: log return of LUATTRUU proxy ---
    ust_tr = load_excel_series(F["spreads_main"], S["ust_index"], C["px_last"])
    ust_tr_rs = resample_level(ust_tr, freq, "last")
    ust_tr_ret = safe_log_return(ust_tr_rs).rename("UST_TR")

    # --- Credit factors: ΔIG OAS, ΔHY OAS, and Δ(HY-IG) ---
    ig_oas = load_excel_series(F["spreads_0109"], S["ig_index"], C["oas"])
    hy_oas = load_excel_series(F["spreads_0109"], S["hy_index"], C["oas"])

    ig_oas_rs = resample_level(ig_oas, freq, "last")
    hy_oas_rs = resample_level(hy_oas, freq, "last")

    d_ig = safe_diff(ig_oas_rs).rename("IG_Credit")
    d_hy = safe_diff(hy_oas_rs).rename("HY_Credit")
    d_stress = safe_diff(hy_oas_rs - ig_oas_rs).rename("Credit_Stress")

    # --- Liquidity factor: aggregate ETF volume shock (safe log return) ---
    # Volume is a flow -> SUM in period; then log-diff
    vol_spy = load_excel_series(F["workshop"], S["spy_vol"], C["volume"])
    vol_hyg = load_excel_series(F["workshop"], S["adj_hyg"], C["volume"])
    vol_lqd = load_excel_series(F["workshop"], S["adj_lqd"], C["volume"])
    vol_tlt = load_excel_series(F["workshop"], S["tlt_vol"], C["volume"])

    vol_spy_rs = resample_level(vol_spy, freq, "sum")
    vol_hyg_rs = resample_level(vol_hyg, freq, "sum")
    vol_lqd_rs = resample_level(vol_lqd, freq, "sum")
    vol_tlt_rs = resample_level(vol_tlt, freq, "sum")

    vol_agg = (vol_spy_rs + vol_hyg_rs + vol_lqd_rs + vol_tlt_rs).rename("VOL_AGG")
    liq = safe_log_return(vol_agg, min_positive=1.0).rename("Liquidity")

    factors = pd.concat(
        [d_10y, ust_tr_ret, vix_change, d_ig, d_hy, d_stress, liq],
        axis=1
    )

    return factors


# ============================================================
# Estimation: OLS per asset (shared X)
# ============================================================

def fit_ols_per_asset(returns: pd.DataFrame, factors: pd.DataFrame) -> dict:
    data = pd.concat([returns, factors], axis=1, join="inner")
    data = clean_for_model(data)

    R = data[["SPY", "HYG", "LQD", "TLT"]]
    X = data.drop(columns=["SPY", "HYG", "LQD", "TLT"])

    Xc = sm.add_constant(X)

    betas = pd.DataFrame(index=["const"] + list(X.columns), columns=R.columns, dtype=float)
    r2 = pd.Series(index=R.columns, dtype=float)
    adj_r2 = pd.Series(index=R.columns, dtype=float)
    models = {}

    for asset in R.columns:
        y = R[asset]
        m = sm.OLS(y, Xc).fit()
        models[asset] = m
        betas[asset] = m.params
        r2.loc[asset] = m.rsquared
        adj_r2.loc[asset] = m.rsquared_adj

    return {
        "data": data,
        "X": X,
        "models": models,
        "betas": betas,
        "r2": r2,
        "adj_r2": adj_r2,
    }


# ============================================================
# PCA on factors + PC regressions per asset
# ============================================================

def fit_pca_factor_model(returns: pd.DataFrame, factors: pd.DataFrame) -> dict:
    data = pd.concat([returns, factors], axis=1, join="inner")
    data = clean_for_model(data)

    R = data[["SPY", "HYG", "LQD", "TLT"]]
    X = data.drop(columns=["SPY", "HYG", "LQD", "TLT"])

    scaler = StandardScaler()
    Xs = scaler.fit_transform(X.values)

    n_comp = min(CONFIG["PCA_MAX_COMPONENTS"], X.shape[1])
    pca = PCA(n_components=n_comp)
    PCs = pca.fit_transform(Xs)

    pc_cols = [f"PC{i+1}" for i in range(PCs.shape[1])]
    PCdf = pd.DataFrame(PCs, index=X.index, columns=pc_cols)

    explained = pd.Series(pca.explained_variance_ratio_, index=pc_cols, name="ExplainedVar")
    loadings = pd.DataFrame(pca.components_.T, index=X.columns, columns=pc_cols)

    K = min(CONFIG["PCA_USE_K"], PCdf.shape[1])
    PC_use = PCdf.iloc[:, :K]
    Xpc = sm.add_constant(PC_use)

    pc_betas = pd.DataFrame(index=["const"] + list(PC_use.columns), columns=R.columns, dtype=float)
    pc_r2 = pd.Series(index=R.columns, dtype=float)
    pc_adj_r2 = pd.Series(index=R.columns, dtype=float)
    pc_models = {}

    for asset in R.columns:
        y = R[asset]
        m = sm.OLS(y, Xpc).fit()
        pc_models[asset] = m
        pc_betas[asset] = m.params
        pc_r2.loc[asset] = m.rsquared
        pc_adj_r2.loc[asset] = m.rsquared_adj

    return {
        "data": data,
        "explained_var": explained,
        "loadings": loadings,
        "pc_models": pc_models,
        "pc_betas": pc_betas,
        "pc_r2": pc_r2,
        "pc_adj_r2": pc_adj_r2,
    }


# ============================================================
# Run joint model for all frequencies
# ============================================================

def run_joint_model_all_frequencies() -> dict:
    portfolio_df = load_excel_df(CONFIG["FILES"]["workshop"], CONFIG["SHEETS"]["portfolio"])

    results = {}

    for label, freq in CONFIG["FREQUENCIES"].items():
        print(f"\n\n==================== {label.upper()} JOINT MODEL ====================")

        # Build
        R = build_asset_returns(portfolio_df, freq=freq)
        F = build_common_factors(freq=freq)

        # Align once (same rows for OLS & PCA) and clean
        data = pd.concat([R, F], axis=1, join="inner")
        data = clean_for_model(data)

        R_aligned = data[["SPY", "HYG", "LQD", "TLT"]]
        F_aligned = data.drop(columns=["SPY", "HYG", "LQD", "TLT"])

        # Fit
        ols_out = fit_ols_per_asset(R_aligned, F_aligned)
        pca_out = fit_pca_factor_model(R_aligned, F_aligned)

        # Quick summaries
        print("\nOLS R² by asset:")
        print(ols_out["r2"].round(4))

        print("\nPCA cumulative explained variance:")
        print(pca_out["explained_var"].cumsum().round(4))

        print("\nPCA R² by asset:")
        print(pca_out["pc_r2"].round(4))

        results[label] = {
            "data": data,
            "returns": R_aligned,
            "factors": F_aligned,
            "ols": ols_out,
            "pca": pca_out,
        }

    return results


# ============================================================
# Execute
# ============================================================

results = run_joint_model_all_frequencies()


# ============================================================
# Summary tables
# ============================================================

def summarize_r2(results_dict: dict) -> pd.DataFrame:
    rows = []
    for freq_label, res in results_dict.items():
        rows.append(pd.Series(res["ols"]["r2"], name=(freq_label, "OLS_R2")))
        rows.append(pd.Series(res["pca"]["pc_r2"], name=(freq_label, "PCA_R2")))
    out = pd.DataFrame(rows)
    out.index = pd.MultiIndex.from_tuples(out.index, names=["Frequency", "Model"])
    return out

summary = summarize_r2(results)
print("\n\n==================== SUMMARY: R² ====================")
try:
    display(summary)
except NameError:
    print(summary)

print("\n\n==================== MONTHLY: OLS BETAS ====================")
try:
    display(results["Monthly"]["ols"]["betas"])
except NameError:
    print(results["Monthly"]["ols"]["betas"])

print("\n\n==================== MONTHLY: PCA LOADINGS ====================")
try:
    display(results["Monthly"]["pca"]["loadings"])
except NameError:
    print(results["Monthly"]["pca"]["loadings"])




==================== DAILY JOINT MODEL ====================

OLS R² by asset:
SPY    0.0078
HYG    0.6750
LQD    0.9059
TLT    0.0108
dtype: float64

PCA cumulative explained variance:
PC1    0.4650
PC2    0.7016
PC3    0.8443
PC4    0.9367
PC5    0.9952
Name: ExplainedVar, dtype: float64

PCA R² by asset:
SPY    0.0030
HYG    0.6651
LQD    0.8755
TLT    0.0019
dtype: float64


==================== WEEKLY JOINT MODEL ====================

OLS R² by asset:
SPY    0.0279
HYG    0.7930
LQD    0.9458
TLT    0.0794
dtype: float64

PCA cumulative explained variance:
PC1    0.4651
PC2    0.7441
PC3    0.8734
PC4    0.9440
PC5    0.9983
Name: ExplainedVar, dtype: float64

PCA R² by asset:
SPY    0.0222
HYG    0.7826
LQD    0.9000
TLT    0.0794
dtype: float64


==================== MONTHLY JOINT MODEL ====================

OLS R² by asset:
SPY    0.4533
HYG    0.9307
LQD    0.9804
TLT    0.5305
dtype: float64

PCA cumulative explained variance:
PC1    0.5072
PC2    0.7489
PC3    0.8784
PC4   

SPY       HYG       LQD       TLT
Frequency Model                                         
Daily     OLS_R2  0.007814  0.675017  0.905915  0.010753
          PCA_R2  0.002957  0.665126  0.875463  0.001870
Weekly    OLS_R2  0.027933  0.793027  0.945844  0.079443
          PCA_R2  0.022166  0.782600  0.900029  0.079396
Monthly   OLS_R2  0.453329  0.930740  0.980383  0.530531
          PCA_R2  0.411652  0.886927  0.971168  0.512563



==================== MONTHLY: OLS BETAS ====================


,SPY,HYG,LQD,TLT
const,0.002193,-0.002800,-0.002625,-0.006165
Delta_10Y,0.146615,-0.051623,-0.059923,-0.066379
UST_TR,3.057235,-0.261046,0.232851,0.568081
VIX_Change,-0.002829,-0.000205,-0.000435,-0.001280
IG_Credit,-0.000737,-0.010739,-0.043653,-0.057621
HY_Credit,-0.021738,-0.019557,-0.022795,-0.015485
Credit_Stress,-0.021001,-0.008818,0.020858,0.042137
Liquidity,0.012376,0.011303,0.003532,0.014776




==================== MONTHLY: PCA LOADINGS ====================


,PC1,PC2,PC3,PC4,PC5
Delta_10Y,0.297645,-0.627111,-0.086380,-0.087329,-0.072689
UST_TR,-0.293654,0.632979,0.063087,0.096381,0.007601
VIX_Change,0.383100,0.125919,-0.363578,0.789696,-0.284440
IG_Credit,0.445259,0.143429,-0.000624,0.072668,0.862355
HY_Credit,0.477207,0.281665,-0.024034,-0.327767,-0.111306
Credit_Stress,0.449161,0.298397,-0.028700,-0.414035,-0.374164
Liquidity,0.225125,-0.035579,0.924645,0.274459,-0.133078
